# Laboratorio N°5 - Computación Científica II - ILI286

**Autores**:
+ Gonzalo Moya - 201173016-k 
+ Ariel Sanhueza     - 201173005-4

## Introducción

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Morbi venenatis dui felis. Donec ante libero, vestibulum nec faucibus sit amet, congue vel erat. Aliquam aliquet dolor mauris. Quisque egestas tincidunt aliquet. Integer tempor tellus quis commodo porttitor. Donec accumsan tellus sed leo sodales, ac malesuada lacus dignissim. Aliquam erat volutpat. Nunc laoreet vitae felis nec varius. Maecenas sed sem non quam faucibus tristique. Nullam scelerisque ante arcu, id faucibus orci tincidunt nec. Aliquam eu leo vehicula leo maximus vulputate. Donec eget nibh dolor. Aliquam eget ullamcorper erat. Nulla sodales, metus finibus venenatis tristique, elit neque pretium mi, eu maximus nibh turpis vulputate lectus. Phasellus vitae pharetra nisi. 

## Desarrollo

### Parte 2:  Diferencias finitas en 2D

### a)  Discretizacion forward

Para la discretizacion con un esquema forward, primero se presentan los elemenos que incluyen la derivación númerica para despues ser reemplazados en los sistemas correspondientes:
\begin{equation*}
{\Delta} u = \frac{u_{i-1,j,n}-2u_{i,j,n}+u_{i+1,j,n}}{{\Delta x}^2} + \frac{u_{i,j-1,n}-2u_{i,j,n}+u_{i,j+1,n}}{{\Delta y}^2}  \\
{\Delta} v = \frac{v_{i-1,j,n}-2v_{i,j,n}+v_{i+1,j,n}}{{\Delta x}^2} + \frac{v_{i,j-1,n}-2v_{i,j,n}+v_{i,j+1,n}}{{\Delta y}^2}  \\
\partial_t u = \frac{u_{i,j,n+1} - u_{i,j,n}}{\Delta t} \\
\partial_t v = \frac{u_{i,j,n+1} - v_{i,j,n}}{\Delta t}.
\end{equation*}
Luego al ser utilizadas estas discretizaciones en el sistema que fue presentado y ademas discretizar el resto de los elemenos que dependan de $u$ y $v$, el restultado es:
\begin{align*}
\frac{u_{i,j,n+1} - u_{i,j,n}}{\Delta t}  = 
D_{u} \frac{u_{i-1,j,n}-2u_{i,j,n}+u_{i+1,j,n}}{{\Delta x}^2} + D_u \frac{u_{i,j-1,n}-2u_{i,j,n}+u_{i,j+1,n}}{{\Delta y}^2} +
\lambda u_{i,j,n} - {u_{i,j,n}}^3 - \sigma v_{i,j,n} \\
\tau \frac{v_{i,j,n+1} - v_{i,j,n}}{\Delta t}  = 
D_{v} \frac{v_{i-1,j,n}-2v_{i,j,n}+v_{i+1,j,n}}{{\Delta x}^2} + D_v \frac{v_{i,j-1,n}-2v_{i,j,n}+v_{i,j+1,n}}{{\Delta y}^2} + u_{i,j,n}  - v_{i,j,n}.
\end{align*}
Finalmente se opera para re-agrupar términos semejantes y dejar ordenado el sistema:
\begin{align*}
u_{i,j,n+1} = \frac{\Delta t D_u}{\Delta x^2}u_{i+1,j,n} + \frac{\Delta t D_u}{\Delta x^2}u_{i-1,j,n} + 
\big( 1 - 2\frac{\Delta t D_u}{\Delta x^2} - 2\frac{\Delta t D_u}{\Delta y^2} + \Delta t \lambda \big)u_{i,j,n} + \Delta t {u_{i,j,k}}^3 \frac{\Delta t D_u}{\Delta y^2}u_{i,j+1,n} + \frac{\Delta t D_u}{\Delta y^2}u_{i,j-1,n} - \sigma v_{i,j,n} \\
\tau v_{i,j,n+1} = \frac{\Delta t D_v}{\Delta x^2}v_{i+1,j,n} + \frac{\Delta t D_v}{\Delta x^2}v_{i-1,j,n} + 
\big( \tau - 2\frac{\Delta t D_u}{\Delta x^2} - 2\frac{\Delta t D_u}{\Delta y^2} - \Delta t \big)v_{i,j,n} +  \frac{\Delta t D_v}{\Delta y^2}v_{i,j+1,n} + \frac{\Delta t D_v}{\Delta y^2}v_{i,j-1,n} + u_{i,j,n} \\ 
\end{align*}


### b)

In [43]:
def solve_forward(u0, v0, dx, dy, dt, Tmax, Du, Dv, l, tau, sigma, k):
    Nx = 50
    Ny = 50
    Nt = Tmax/dt
    
    # Puntos
    x = np.linspace(0, 1, Nx)
    y = np.linspace(0, 1, Ny)
    t = np.linspace(0, Tmax, Nt)
    print len(x),len(y),len(t)
    
    # Soluciones
    sol_u = np.zeros((Nt, Nx, Ny))
    sol_v = np.zeros((Nt, Nx, Ny))
    
    # Valores constantes
    a_u = dt*Du/(dx**2)
    b_u = dt*Du/(dy**2)
    a_v = dt*Dv/(dx**2)
    b_v = dt*Dv/(dy**2)
    
    # condiciones iniciales u/v
    sol_u[0,:,:] = u0
    sol_v[0,:,:] = v0
    
    for n in xrange(1, len(t)):
        # Resto de casos
        Cu = np.zeros((sol_u.shape[1]+2, sol_u.shape[2]+2))
        Cu[1:-1,1:-1] = sol_u[n-1,:,:]
        Cu[-1,1:-1] = sol_u[n-1,1,:]
        Cu[0,1:-1] = sol_u[n-1,-2,:]
        Cu[1:-1,0] = sol_u[n-1,:,-2]
        Cu[1:-1,-1] = sol_u[n-1,:,1]

        Cv = np.zeros((sol_u.shape[1]+2, sol_u.shape[2]+2))
        Cv[1:-1,1:-1] = sol_v[n-1,:,:]
        Cv[-1,1:-1] = sol_v[n-1,1,:]
        Cv[0,1:-1] = sol_v[n-1,-2,:]
        Cv[1:-1,0] = sol_v[n-1,:,-2]
        Cv[1:-1,-1] = sol_v[n-1,:,1]

        for i in xrange(0, len(x)):
            for j in xrange(0, len(y)):
                sol_u[n,i,j] = a_u*Cu[i+1,j]+a_u*Cu[i-1,j]+(1-2*a_u-2*b_u+dt*l)*Cu[i,j]+b_u*Cu[i,j+1]+b_u*Cu[i,j-1]-dt*sigma*Cv[i,j]-dt*k-dt*(Cu[i,j]**3)
                sol_v[n,i,j] = a_v*Cv[i+1,j]+a_v*Cv[i-1,j]+(tau-2*a_v-2*b_v-dt)*Cv[i,j]+b_v*Cv[i,j+1]+b_v*Cv[i,j-1]+dt*Cu[i,j]
                sol_v[n,i,j] = sol_v[n,i,j]/tau
                if math.isnan(sol_u[n,i,j]):
                    print n,i,j
                    return sol_u
    
    return sol_u

### c)

### d)

# Parte 4: Simulación

In [56]:
dt = 0.0005
dx = dy = 0.05
u0 = np.random.rand(50,50)
u0[:,0] = u0[:,-1] = np.random.rand(50)
u0[0,:] = u0[-1,:] = np.random.rand(50)

v0 = np.random.rand(50,50)
v0[:,0] = v0[:,-1] = np.random.rand(50)
v0[0,:] = v0[-1,:] = np.random.rand(50)
Du = 3*np.e - 4
Dv = 4*np.e - 3
l = 1.0
tau = 0.2
sigma = 1.0
k = -0.004
sol = solve_forward(u0, v0, dx, dy, dt, 10, Du, Dv, l, tau, sigma, k)
print sol
#animate.animate_pattern(sol)

50 50 20000
11 0 4
[[[ 0.47133797  0.65953225  0.4055382  ...,  0.53024652  0.05103274
    0.68755688]
  [ 0.50676999  0.14145396  0.46478258 ...,  0.56008323  0.62452656
    0.50676999]
  [ 0.19962618  0.34569011  0.97509837 ...,  0.38331036  0.15912303
    0.19962618]
  ..., 
  [ 0.98718208  0.06660571  0.29708668 ...,  0.20204351  0.90052994
    0.98718208]
  [ 0.42564581  0.4443282   0.32284233 ...,  0.48200134  0.83355156
    0.42564581]
  [ 0.47133797  0.65953225  0.4055382  ...,  0.53024652  0.05103274
    0.68755688]]

 [[ 0.39610715  0.19265844  0.25500941 ...,  1.20094531  0.69631644
   -0.62174375]
  [ 1.33993381  0.26975831 -0.31733713 ..., -0.16694306  0.1683617
    2.10482818]
  [-0.73807307  0.01653907  1.31360513 ...,  0.66800048  0.34746731
   -0.39020508]
  ..., 
  [ 0.53668297  0.52134011  0.86984786 ..., -1.03552569  2.00638695
    0.24424191]
  [-0.00903702 -0.66064319  1.67849223 ...,  0.62028136  1.13649419
    0.10350762]
  [-0.42369869  1.28452163  0.19281283 .

/usr/lib/python2.7/site-packages/IPython/kernel/__main__.py:44: RuntimeWarning: overflow encountered in double_scalars
/usr/lib/python2.7/site-packages/IPython/kernel/__main__.py:44: RuntimeWarning: invalid value encountered in double_scalars


In [53]:
animate.animate_pattern(sol, time_step=5)

True


## Conclusiones

+ Nullam tincidunt commodo arcu et consectetur. Duis iaculis, risus vel scelerisque interdum, ante nunc varius urna, nec accumsan ex purus vitae turpis. Etiam ut egestas ligula, id lacinia nunc. 
+ Praesent sodales ligula eu lorem placerat convallis. Maecenas elementum venenatis maximus. Class aptent taciti sociosqu ad litora torquent per conubia nostra, per inceptos himenaeos. Proin vitae finibus nulla, vel auctor orci. Ut et sodales lectus, nec sodales nunc. 
+ Quisque laoreet feugiat diam, a lacinia felis pharetra eu. Donec feugiat elit sapien, a cursus diam hendrerit in.
+ Etiam porta ipsum a aliquet hendrerit. Pellentesque fringilla dolor pulvinar lorem ullamcorper, non vestibulum nulla volutpat. Sed ac diam fringilla tortor iaculis tincidunt. 
+ Integer sapien augue, accumsan nec sagittis at, sagittis quis quam. Maecenas consectetur metus non tellus laoreet, nec efficitur lacus venenatis. Nullam ac dolor quis orci pretium commodo. 
+ Maecenas eget ex in lectus tincidunt lobortis. Donec non gravida nisl. Fusce sagittis sagittis neque consectetur cursus. Duis commodo mauris ligula, eu aliquam purus porta quis. 
+ Suspendisse euismod mi lorem, in ultricies neque interdum sit amet. Vivamus ut ullamcorper tortor, vel faucibus risus. Curabitur malesuada lorem diam. Nam ornare nisi id sapien viverra, semper commodo dolor eleifend. 

## Anexos

En la siguiente sección se encuentra todo el código necesario para reproducir cada uno de los resultados mostrados anteriormente. Para poder ejecutar el código en el informe, se debe en primer lugar ejecutar las celdas de código presentes en este anexo.

### Configuración del notebook

In [29]:
#notebook settings
#%matplotlib inline
%matplotlib qt

#import some useful libraries and utilities
import numpy as np
import matplotlib.pyplot as plt
import animate
import math